In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
!pip install pretrainedmodels

import os
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import pretrainedmodels
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_img_dir = "/kaggle/input/isic-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data"
train_csv = "/kaggle/input/isic-2017/ISIC-2017_Training_Part3_GroundTruth.csv"

val_img_dir = "/kaggle/input/isic-2017/ISIC-2017_Validation_Data/ISIC-2017_Validation_Data"
val_csv = "/kaggle/input/isic-2017/ISIC-2017_Validation_Part3_GroundTruth.csv"

num_classes = 3
batch_size = 32
num_epochs = 15
lr = 1e-4


In [14]:
class ISICDataset(Dataset):
    def __init__(self, img_dir, csv_file, transform=None):
        self.img_dir = img_dir
        self.df = pd.read_csv(csv_file)
        self.transform = transform

        self.df = self.df[self.df['image_id'].apply(lambda x: os.path.exists(os.path.join(img_dir, x + ".jpg")))]

        # Compute nevus = 1 - (melanoma + SK)
        self.df['nevus'] = 1 - (self.df['melanoma'] + self.df['seborrheic_keratosis'])
        self.labels = self.df[['melanoma', 'seborrheic_keratosis', 'nevus']].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]['image_id']
        img_path = os.path.join(self.img_dir, img_id + ".jpg")

        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        label = torch.tensor(np.argmax(label)).long()

        if self.transform:
            image = self.transform(image)

        return image, label


In [15]:
transform_train = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

transform_val = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dataset = ISICDataset(train_img_dir, train_csv, transform=transform_train)
val_dataset = ISICDataset(val_img_dir, val_csv, transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [16]:
model = pretrainedmodels.__dict__['xception'](num_classes=1000, pretrained='imagenet')
model.last_linear = nn.Linear(model.last_linear.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [19]:
best_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_labels, val_preds)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_xception.pth")
        print("✅ Saved best model!")


  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/15, Train Loss: 0.6755, Val Acc: 0.7333
✅ Saved best model!


  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 5/15, Train Loss: 0.0855, Val Acc: 0.7667


  0%|          | 0/125 [00:00<?, ?it/s]

KeyboardInterrupt: 